In [ ]:
import tensorflow as tf
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
import pathlib

In [ ]:
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file(origin=dataset_url,
                                   fname='flower_photos',
                                   untar=True)
data_dir = pathlib.Path(data_dir)

In [ ]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255.0,
                   validation_split=0.2,
                   dtype="float32")

In [ ]:
b_size = 4
img_size = 180
trainset = datagen.flow_from_directory(data_dir,target_size=(img_size,img_size),color_mode='rgb',subset="training",batch_size=b_size,class_mode="categorical")
valset = datagen.flow_from_directory(data_dir,target_size=(img_size,img_size),color_mode='rgb',subset="validation",batch_size=b_size,class_mode="categorical")
print(trainset.class_indices)

# Finetuning
![finetune](finetune.png)

In [ ]:
vgg_model = tf.keras.applications.VGG16(weights="imagenet",include_top=False, input_shape = (img_size,img_size,3))

In [ ]:
for layer in vgg_model.layers:
    layer.trainable = False
    print(layer)

In [ ]:
x = vgg_model.output
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(100, activation="relu")(x)
x = tf.keras.layers.Dense(100, activation="relu")(x)
x = tf.keras.layers.Dense(5, activation='softmax')(x)
transfered_model = tf.keras.Model(inputs = vgg_model.input, outputs = x)
transfered_model.summary()

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
transfered_model.compile(optimizer=optimizer, loss = "categorical_crossentropy", metrics=["accuracy"])

In [ ]:
transfered_model.fit(trainset,verbose=1,validation_data=valset,batch_size=8,epochs=10)